In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('bikeshare_2023.csv', low_memory=False)

In [ ]:
# Check info and dtypes of the dataframe
print("Info of the DataFrame:\n", '\n')
df.info()

In [ ]:
unique_values_before = df.nunique()
print("Unique values before conversion:\n", unique_values_before, '\n')

Potentially execute some data cleaning at this stage!

In [ ]:
# Create a new dataset with data for all of 2023

# weekend yes / no
# --> have for each season option to further differentiate between weekend yes, or no

# aggregate by season (again 90 days balance) 
# --> allows me to see changes from one season to the next
# --> balancing only problem in certain seasons (summer!?)
# --> have menu to shift between seasons (starting point: summer or whichever season effect is the strongest)

# then integrate into analysis member-type
# --> have for each season option to further differentiate between all, casual-only, member-only

# disregard electric / classic bike

# if I still have time
# what is close-by at main underbalance or overbalance location
# commuter station locations along main roads to north do no seem to have a problem (members only, same day return!?)


In [ ]:
# Compare to capacity 
# make calculation for high-frequency stations 


DATA w/o member type

In [ ]:
# exclude docked bikes if the column exists
if 'rideable_type' in df.columns:
    df = df[df['rideable_type'] != 'docked_bike']

# check count of start stations per day
df_2 = df.copy() # to make sure any changes in df_full will not automatically affect df_full_2

# Create a new dataframe with unique dates and stations
unique_dates = df_2['date'].unique()
unique_stations = df_2['end_station_name'].unique()

# Extract unique end_station_name with their corresponding latitudes and longitudes
unique_stations_lat_lng = df_2[['end_station_name', 'end_lat', 'end_lng']].drop_duplicates().rename(columns={'end_station_name': 'STATION', 'end_lat': 'LAT', 'end_lng': 'LNG'})

# Create df_2_unique with unique dates and stations
df_2_unique = pd.MultiIndex.from_product([unique_dates, unique_stations], names=['DATE', 'STATION']).to_frame(index=False)

# Add the 'weekday' column directly behind each date
df_2_unique = df_2_unique.merge(df_2[['date', 'weekday']].drop_duplicates(), left_on='DATE', right_on='date', how='left', suffixes=('', '_weekday'))
df_2_unique.rename(columns={'weekday': 'WEEKDAY'}, inplace=True)

# Add the 'season' column directly behind each date
df_2_unique = df_2_unique.merge(df_2[['date', 'season']].drop_duplicates(), left_on='DATE', right_on='date', how='left', suffixes=('', '_season'))
df_2_unique.rename(columns={'season': 'SEASON'}, inplace=True)

# Merge the latitudes and longitudes into df_2_unique
df_2_unique = df_2_unique.merge(unique_stations_lat_lng, left_on='STATION', right_on='STATION', how='left')

# Calculate COUNT_PICKUP and COUNT_RETURN
count_pickup = df_2.groupby(['date', 'start_station_name']).size().reset_index(name='COUNT_PICKUP')
count_return = df_2.groupby(['date', 'end_station_name']).size().reset_index(name='COUNT_RETURN')

# Merge the counts into df_2_unique
df_2_unique = df_2_unique.merge(count_pickup, left_on=['DATE', 'STATION'], right_on=['date', 'start_station_name'], how='left', suffixes=('', '_pickup'))
df_2_unique = df_2_unique.merge(count_return, left_on=['DATE', 'STATION'], right_on=['date', 'end_station_name'], how='left', suffixes=('', '_return'))

# Fill NaN values with 0
df_2_unique['COUNT_PICKUP'] = df_2_unique['COUNT_PICKUP'].fillna(0)
df_2_unique['COUNT_RETURN'] = df_2_unique['COUNT_RETURN'].fillna(0)

# Simplify the overview to four columns
df_2_unique = df_2_unique[['DATE', 'WEEKDAY', 'SEASON', 'STATION', 'LAT', 'LNG', 'COUNT_PICKUP', 'COUNT_RETURN']]

# Add a new column called 'BALANCE' at the end which by line subtracts COUNT_RETURN from COUNT_PICKUP
df_2_unique['BALANCE'] = df_2_unique['COUNT_RETURN'] - df_2_unique['COUNT_PICKUP']

# Remove all rows with empty value in column 'STATION'
df_2_unique = df_2_unique.dropna(subset=['STATION'])

# Add the 'week_number' column, rename it to 'WEEK_NR', and place it behind the 'DATE' column
df_2_unique = df_2_unique.merge(df_2[['date', 'week_number']].drop_duplicates(), left_on='DATE', right_on='date', how='left')
df_2_unique.rename(columns={'week_number': 'WEEK_NR'}, inplace=True)
df_2_unique = df_2_unique[['DATE', 'WEEK_NR', 'WEEKDAY', 'SEASON', 'STATION', 'LAT', 'LNG', 'COUNT_PICKUP', 'COUNT_RETURN', 'BALANCE']]

df_2_unique

# Questions: 3000 rows were added when LAT and LNG were added as columns

In [ ]:
# Aggregate by station and week number the cumulative BALANCE, COUNT_PICKUP, and COUNT_RETURN for all days
df_cumulative_balance = df_2_unique.groupby(['STATION', 'WEEK_NR']).agg({
    'BALANCE': 'sum',
    'COUNT_PICKUP': 'sum',
    'COUNT_RETURN': 'sum'
}).reset_index()

# Rename the BALANCE column to CUMULATIVE_BALANCE
df_cumulative_balance.rename(columns={'BALANCE': 'CUMULATIVE_BALANCE'}, inplace=True)

# Reorder columns to place 'CUMULATIVE_BALANCE' as the last column
df_cumulative_balance = df_cumulative_balance[['STATION', 'WEEK_NR', 'COUNT_PICKUP', 'COUNT_RETURN', 'CUMULATIVE_BALANCE']]

# Sort the results by STATION and WEEK_NR
df_cumulative_balance = df_cumulative_balance.sort_values(by=['STATION', 'WEEK_NR'])

df_cumulative_balance

In [ ]:
# Filter the DataFrame for the specified station
station_records = df_cumulative_balance[df_cumulative_balance['STATION'] == '11th & Kenyon St NW']

# Display the records
station_records

In [ ]:
# Filter the DataFrame for the specified station
station_records = df_cumulative_balance[df_cumulative_balance['STATION'] == '14th St & Rhode Island Ave NW']

# Display the records
station_records

In [ ]:
# '14th & Rhode Island Ave NW'
# Filter the DataFrame for the specified station
station_records = df_cumulative_balance[df_cumulative_balance['STATION'] == '14th & Rhode Island Ave NW']

# Display the records
station_records

In [ ]:
# Filter the DataFrame for the two stations
stations_to_merge = ['14th & Rhode Island Ave NW', '14th St & Rhode Island Ave NW']
filtered_stations = df_cumulative_balance[df_cumulative_balance['STATION'].isin(stations_to_merge)]

# Group by WEEK_NR and sum the values for the two stations
merged_values = filtered_stations.groupby('WEEK_NR').agg({
    'COUNT_PICKUP': 'sum',
    'COUNT_RETURN': 'sum',
    'CUMULATIVE_BALANCE': 'sum'
}).reset_index()

# Add the merged values to the station '14th St & Rhode Island Ave NW'
merged_values['STATION'] = '14th St & Rhode Island Ave NW'

# Remove the original records for the two stations
df_cumulative_balance = df_cumulative_balance[~df_cumulative_balance['STATION'].isin(stations_to_merge)]

# Append the merged values back to the DataFrame
df_cumulative_balance = pd.concat([df_cumulative_balance, merged_values], ignore_index=True)

# Ensure 'LAT' and 'LNG' values for '14th St & Rhode Island Ave NW' are retained
df_cumulative_balance.loc[df_cumulative_balance['STATION'] == '14th St & Rhode Island Ave NW', ['LAT', 'LNG']] = [first_lat, first_lng]

# Delete all rows for station '14th & Rhode Island Ave NW'
df_cumulative_balance = df_cumulative_balance.drop(df_cumulative_balance[df_cumulative_balance['STATION'] == '14th & Rhode Island Ave NW'].index)

# Filter and display the updated DataFrame for '14th St & Rhode Island Ave NW'
df_cumulative_balance[df_cumulative_balance['STATION'] == '14th St & Rhode Island Ave NW']

In [ ]:
# Check for different values in 'LAT' or 'LNG' for the same 'STATION'
lat_lng_check = df_2_unique.groupby('STATION').agg({'LAT': pd.Series.nunique, 'LNG': pd.Series.nunique})
stations_with_multiple_lat_lng = lat_lng_check[(lat_lng_check['LAT'] > 1) | (lat_lng_check['LNG'] > 1)].index

# Amend all records for the same 'STATION' to the first found values
for station in stations_with_multiple_lat_lng:
    first_lat = df_2_unique.loc[df_2_unique['STATION'] == station, 'LAT'].dropna().iloc[0]
    first_lng = df_2_unique.loc[df_2_unique['STATION'] == station, 'LNG'].dropna().iloc[0]
    df_2_unique.loc[df_2_unique['STATION'] == station, 'LAT'] = first_lat
    df_2_unique.loc[df_2_unique['STATION'] == station, 'LNG'] = first_lng

df_cumulative_balance = df_cumulative_balance.merge(df_2_unique[['STATION', 'WEEK_NR', 'LAT', 'LNG', 'COUNT_PICKUP', 'COUNT_RETURN']].drop_duplicates(), on='STATION', how='left')
df_cumulative_balance = df_2_unique.groupby(['STATION', 'WEEK_NR', 'LAT', 'LNG'])[['COUNT_PICKUP', 'COUNT_RETURN', 'BALANCE']].sum().reset_index().sort_values(by='BALANCE')
df_cumulative_balance.rename(columns={'BALANCE': 'CUMULATIVE_BALANCE'}, inplace=True)
df_cumulative_balance = df_cumulative_balance[['STATION', 'WEEK_NR', 'LAT', 'LNG', 'COUNT_PICKUP', 'COUNT_RETURN', 'CUMULATIVE_BALANCE']]
df_cumulative_balance

In [ ]:
# Generate a histogram plot using seaborn
plt.figure(figsize=(14, 8))
sns.histplot(data=df_cumulative_balance, x='CUMULATIVE_BALANCE', bins=30, kde=True)

# Set the title and labels
plt.title('Weekly distribution of Cumulative Balance by Station')
plt.xlabel('Cumulative Balance')
plt.ylabel('Count of Weekly Station Records')

# Add values on top of each bar, further away from the top end
for p in plt.gca().patches:
    plt.gca().annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height() + 5.0), ha='center', va='baseline')

# Show the plot
plt.show()

In [ ]:
# df_cumulative_balance[df_cumulative_balance['CUMULATIVE_BALANCE'] > 450]
# equates to 111 rows if criteria is set to +90 (= +1/day over 90 days)
# equates to 72 rows if criteria is set to +180 (= +2/day over 90 days)
# equates to 55 rows if criteria is set to +270 (= +3/day over 90 days)
# equates to 46 rows if criteria is set to +360 (= +4/day over 90 days)
# equates to 32 rows if criteria is set to +450 (= +5/day over 90 days)


df_cumulative_balance[df_cumulative_balance['CUMULATIVE_BALANCE'] < -450]
# equates to 111 rows if criteria is set to -90 (= -1/day over 90 days)
# equates to 72 rows if criteria is set to -180 (= -2/day over 90 days)
# equates to 53 rows if criteria is set to -270 (= -3/day over 90 days)
# equates to 37 rows if criteria is set to -360 (= -4/day over 90 days)
# equates to 27 rows if criteria is set to -450 (= -5/day over 90 days)

In [ ]:
# import csv file capacity_data.csv
df_capacity = pd.read_csv('capacity_data.csv', sep=';')
df_capacity.tail()

In [ ]:
df_cumulative_balance = df_cumulative_balance.drop(df_cumulative_balance[df_cumulative_balance['STATION'] == '14th & Rhode Island Ave NW'].index)

# compare row values in column start_station_name of dataframe df_capacity with row values in column STATION of dataframe df_cumulative_balance and print the mis-matches
# Compare STATION in df_cumulative_balance with start_station_name in df_capacity and print mismatches
mismatches = df_cumulative_balance[~df_cumulative_balance['STATION'].isin(df_capacity['start_station_name'])]
# print("Mismatches:\n", mismatches)
mismatches

In [ ]:
# Similar to code section above execute code to show both stations only in df_cumulative
# filtered_stations1 = df_cumulative_balance[df_cumulative_balance['STATION'].isin(['3rd & M St SE', '3rd & M St NE'])]
# 14 min apart,i.e. not same stations
# filtered_stations1 = df_cumulative_balance[df_cumulative_balance['STATION'].isin(['3rd & N St NE', '3rd & M St NE'])]
# filtered_stations1

In [ ]:
df_cumulative_balance

MERGE CAPACITY DATA WITH CUMULATIVE BALANCE

In [ ]:
# Merge df_cumulative_balance with df_capacity on the station name
df_cumulative_balance = df_cumulative_balance.merge(df_capacity, left_on='STATION', right_on='start_station_name', how='left')

# Rename the 'capacity' column to 'CAPACITY'
df_cumulative_balance.rename(columns={'capacity': 'CAPACITY'}, inplace=True)

# Drop the redundant 'start_station_name' column
df_cumulative_balance.drop(columns=['start_station_name'], inplace=True)

# Reorder columns to keep 'COUNT_PICKUP' and 'COUNT_RETURN' behind 'LNG'
columns_order = ['STATION', 'WEEK_NR', 'LAT', 'LNG', 'COUNT_PICKUP', 'COUNT_RETURN', 'CUMULATIVE_BALANCE', 'CAPACITY']
df_cumulative_balance = df_cumulative_balance[columns_order]

df_cumulative_balance


In [ ]:
# Count the number of stations for each capacity value
capacity_counts = df_cumulative_balance['CAPACITY'].value_counts().sort_index()

# Create a pie chart
plt.figure(figsize=(10, 8))
plt.pie(capacity_counts, labels=capacity_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Stations by Capacity')
plt.ylabel('Count of Stations')
plt.show()

Used OpenOfficeCalc to see that 87% stations have capacity up to 21days, 15 days limit would only cover 56%

Development of calculation logic for imbalance categories:
1. Imbalance starts when whole station is full or empty
2. Hence, if I set limit at 21 bikes (+/-) then I have an accurate measurement (covers 87% of stations)
3. Even better way would be to calculate imbalance for each station capacity category individually () 

INTRODUCTION OF IMBALANCE FACTOR

In [ ]:
# calculate an 'IMBALANCE FACTOR' for each row in df_cumulative_balance. Take value in 'CUMULATIVE_BALANCE' (weekly), 
# then take the result and divide it by the value in column 'CAPACITY' in same. 
# Result of this calculation should be put as float with one decimal in the new column 'IMBALANCE FACTOR' 
df_cumulative_balance['IMBALANCE_FACTOR'] = (df_cumulative_balance['CUMULATIVE_BALANCE'] / df_cumulative_balance['CAPACITY']).round(1)
df_cumulative_balance

In [ ]:
df_cumulative_balance['IMB_FAC_CATEGORY'] = df_cumulative_balance['IMBALANCE_FACTOR'].apply(
    lambda x: 'too_few' if x < -0.5 else ('balanced' if x <= 0.5 else 'too_many')
)
df_cumulative_balance

In [ ]:
# Define the color scheme
# Count the number of stations in each IMB_FAC_CATEGORY
imb_fac_category_counts = df_cumulative_balance['IMB_FAC_CATEGORY'].value_counts()

# Plot the distribution as a bar chart
plt.figure(figsize=(10, 6))
ax = plt.gca()
bars = ax.bar(imb_fac_category_counts.index, imb_fac_category_counts.values)

# Apply colors to each bar based on its category
for bar, category in zip(bars, imb_fac_category_counts.index):
    color_scheme = {'balanced': 'lightgreen', 'too_few': 'lightcoral', 'too_many': 'lightblue'}
    bar.set_color(color_scheme[category])

plt.title('Distribution of Imbalance-Factor categories')
plt.xlabel('Imbalance-Factor category')
plt.ylabel('Count of Stations')
plt.xticks(rotation=45)

# Add values on top of each bar, further away from the top end
for p in plt.gca().patches:
    plt.gca().annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height() + 5.0), ha='center', va='baseline')

plt.show()

In [ ]:
# Count the number of stations in each IMB_FAC_CATEGORY
imb_fac_category_counts = df_cumulative_balance['IMB_FAC_CATEGORY'].value_counts()

# Plot the distribution as a pie chart
plt.figure(figsize=(10, 6))
imb_fac_category_counts.plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=['lightgreen', 'lightcoral', 'lightblue'])
plt.title('Distribution of Imbalance-Factor categories')
plt.ylabel('')  # Hide the y-label

# Add a legend with the counts
legend_labels = [f'{category}: {count}' for category, count in imb_fac_category_counts.items()]
plt.legend(legend_labels, loc='upper left', bbox_to_anchor=(1, 0.5))

plt.show()


INTRODUCTION OF TURNOVER

In [ ]:
# introduce new calculation
df_cumulative_balance['TURNOVER'] = ((df_cumulative_balance['COUNT_PICKUP'] / 7) / df_cumulative_balance['CAPACITY']).round(1)
df_cumulative_balance



In [ ]:
# create a histplot using count of stations on y-axis and turnover on x-axis
plt.figure(figsize=(12, 6))
sns.histplot(data=df_cumulative_balance, x='TURNOVER', bins=30, kde=True)
plt.xlabel('Turnover')
plt.ylabel('Count of Stations')
plt.title('Distribution of Turnover Across Stations')

# Add values on top of each bar, further away from the top end
for p in plt.gca().patches:
    plt.gca().annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height() + 5.0), ha='center', va='baseline')

plt.show()

In [ ]:
# Filter the dataframe to include only records with TURNOVER >= 1.5
df_filtered = df_cumulative_balance[df_cumulative_balance['TURNOVER'] >= 1.5]

# Create a histplot using count of stations on y-axis and turnover on x-axis
plt.figure(figsize=(12, 6))
sns.histplot(data=df_filtered, x='TURNOVER', bins=30, kde=True)
plt.xlabel('Turnover')
plt.ylabel('Count of Stations')
plt.title('Distribution of Turnover Across Stations (TURNOVER >= 1.5)')

# Add values on top of each bar, further away from the top end
for p in plt.gca().patches:
    plt.gca().annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height() + 1.0), ha='center', va='baseline')

plt.show()

In [ ]:
def categorize_turnover(turnover):
    if turnover > 2:
        return 'highly-utilized'
    elif 0.33 <= turnover <= 2:
        return 'well-utilized'
    else:
        return 'under-utilized'

df_cumulative_balance['TO_CATEGORY'] = df_cumulative_balance['TURNOVER'].apply(categorize_turnover)
df_cumulative_balance


In [ ]:
# Define the order and color scheme
category_order = ['highly-utilized', 'well-utilized', 'under-utilized']
category_colors = {
    'highly-utilized': 'darkgreen',
    'well-utilized': 'lightgreen',
    'under-utilized': 'lightblue',
}

# Count the number of stations in each turnover category
category_counts = df_cumulative_balance['TO_CATEGORY'].value_counts().reindex(category_order)

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%', startangle=140, colors=[category_colors[cat] for cat in category_order])
plt.title('Distribution of Stations by Turnover Category')

# Add a legend with the counts
legend_labels = [f'{category}: {count}' for category, count in category_counts.items()]
plt.legend(legend_labels, loc='upper left', bbox_to_anchor=(1, 0.5))

plt.show()

INTRODUCTION OF CROSS-CATEGORIES

In [ ]:
# build cross-category
df_cumulative_balance['TO_IMB_CATEGORY'] = df_cumulative_balance['TO_CATEGORY'] + '__' + df_cumulative_balance['IMB_FAC_CATEGORY']
df_cumulative_balance

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define the categories for the x and y axes
to_categories = ['under-utilized', 'well-utilized', 'highly-utilized']
imb_fac_categories = ['too_few', 'balanced', 'too_many']

# Create a DataFrame to store the counts
counts = pd.DataFrame(0, index=imb_fac_categories, columns=to_categories)

# Count the number of stations in each cross-category
for to_cat in to_categories:
    for imb_fac_cat in imb_fac_categories:
        counts.loc[imb_fac_cat, to_cat] = df_cumulative_balance[
            (df_cumulative_balance['TO_CATEGORY'] == to_cat) & 
            (df_cumulative_balance['IMB_FAC_CATEGORY'] == imb_fac_cat)
        ].shape[0]

# Create the plot
fig, ax = plt.subplots(figsize=(6, 6))

# Define the color scheme for each quadrant
color_scheme = {
    ('too_many', 'under-utilized'): 'lightblue',
    ('too_many', 'well-utilized'): 'blue',
    ('too_many', 'highly-utilized'): 'darkblue',
    ('balanced', 'under-utilized'): 'lightgreen',
    ('balanced', 'well-utilized'): 'green',
    ('balanced', 'highly-utilized'): 'darkgreen',
    ('too_few', 'under-utilized'): 'lightcoral',
    ('too_few', 'well-utilized'): 'red',
    ('too_few', 'highly-utilized'): 'darkred'
}

# Create a scatter plot with circle sizes based on the counts and fixed color scheme
for i, imb_fac_cat in enumerate(imb_fac_categories):
    for j, to_cat in enumerate(to_categories):
        count = counts.loc[imb_fac_cat, to_cat]
        color = color_scheme.get((imb_fac_cat, to_cat), 'white')  # Use 'white' for no values
        ax.scatter(j, i, s=count*10*2.0, alpha=0.6, edgecolors='w', color=color)  # Increase size by 25%
        ax.text(j, i, count, ha='center', va='center', color='black')

# Set the x and y axis labels
ax.set_xticks(range(len(to_categories)))
ax.set_xticklabels(to_categories)
ax.set_yticks(range(len(imb_fac_categories)))
ax.set_yticklabels(imb_fac_categories)

# Set the axis labels
ax.set_xlabel('Turnover')
ax.set_ylabel('Imbalance')

# Set the title
ax.set_title('Station categorization\n based on Turnover & Imbalance')

# Adjust the x and y limits to add padding
ax.set_xlim(-0.5, len(to_categories) - 0.5)
ax.set_ylim(-0.5, len(imb_fac_categories) - 0.5)

plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define the categories for the x and y axes
to_categories = ['under-utilized', 'well-utilized', 'highly-utilized']
imb_fac_categories = ['too_few', 'balanced', 'too_many']

# Create a DataFrame to store the counts
counts = pd.DataFrame(0, index=imb_fac_categories, columns=to_categories)

# Count the number of stations in each cross-category
for to_cat in to_categories:
    for imb_fac_cat in imb_fac_categories:
        counts.loc[imb_fac_cat, to_cat] = df_cumulative_balance[
            (df_cumulative_balance['TO_CATEGORY'] == to_cat) & 
            (df_cumulative_balance['IMB_FAC_CATEGORY'] == imb_fac_cat)
        ].shape[0]

# Create the plot
fig, ax = plt.subplots(figsize=(6, 6))

# Define the color scheme for each quadrant
color_scheme = {
    ('too_many', 'under-utilized'): 'lightblue',
    ('too_many', 'well-utilized'): 'blue',
    ('too_many', 'highly-utilized'): 'darkblue',
    ('balanced', 'under-utilized'): 'lightgreen',
    ('balanced', 'well-utilized'): 'green',
    ('balanced', 'highly-utilized'): 'darkgreen',
    ('too_few', 'under-utilized'): 'lightcoral',
    ('too_few', 'well-utilized'): 'red',
    ('too_few', 'highly-utilized'): 'darkred'
}

# Create a scatter plot with circle sizes based on the counts and fixed color scheme
for i, imb_fac_cat in enumerate(imb_fac_categories):
    for j, to_cat in enumerate(to_categories):
        count = counts.loc[imb_fac_cat, to_cat]
        color = color_scheme.get((imb_fac_cat, to_cat), 'white')  # Use 'white' for no values
        ax.scatter(j, i, s=count*10*2.0, alpha=0.6, edgecolors='w', color=color)  # Increase size by 25%
        ax.text(j, i, count, ha='center', va='center', color='black')

# Set the x and y axis labels
ax.set_xticks(range(len(to_categories)))
ax.set_xticklabels(to_categories)
ax.set_yticks(range(len(imb_fac_categories)))
ax.set_yticklabels(imb_fac_categories)

# Set the axis labels
ax.set_xlabel('Turnover')
ax.set_ylabel('Imbalance')

# Set the title
ax.set_title('Station categorization\n based on Turnover & Imbalance')

# Adjust the x and y limits to add padding
ax.set_xlim(-0.5, len(to_categories) - 0.5)
ax.set_ylim(-0.5, len(imb_fac_categories) - 0.5)

plt.show()

In [ ]:
import folium
from IPython.display import display, clear_output
import time

# Create a base map
m = folium.Map(location=[df_cumulative_balance['LAT'].mean(), df_cumulative_balance['LNG'].mean()], zoom_start=12)

# Function to determine color based on TO_IMB_CATEGORY
def get_color(to_imb_category):
    return color_scheme.get(to_imb_category, 'blue')

# Function to update the map for a specific week
def update_map(week_nr):
    # Clear all markers from the map
    m = folium.Map(location=[df_cumulative_balance['LAT'].mean(), df_cumulative_balance['LNG'].mean()], zoom_start=12) # Use zoom = 13 if you want to see the stations in more detail
    
    
    # Add stations to the map for the given week
    week_data = df_cumulative_balance[df_cumulative_balance['WEEK_NR'] == week_nr]
    for _, row in week_data.iterrows():
        folium.CircleMarker(
            location=[row['LAT'], row['LNG']],
            radius=5 + abs(row['IMBALANCE_FACTOR']) * 5,  # Scale the size based on IMBALANCE_FACTOR
            color=get_color((row['IMB_FAC_CATEGORY'], row['TO_CATEGORY'])),
            fill=True,
            fill_color=get_color((row['IMB_FAC_CATEGORY'], row['TO_CATEGORY'])),
            fill_opacity=0.6,
            popup=f"Station: {row['STATION']}\nTurnover: {row['TURNOVER']:.2f}\nImbalance Factor: {row['IMBALANCE_FACTOR']:.2f}"
        ).add_to(m)
    
    # Add a progress display
    progress_html = f'''
    <div style="position: fixed; 
                top: 10px; left: 50%; transform: translateX(-50%); 
                width: 200px; height: 30px; 
                border:2px solid grey; z-index:9999; font-size:16px;
                background-color:white; opacity: 0.8;
                text-align: center; line-height: 30px;">
    Week {week_nr} / 52
    </div>
    '''
    m.get_root().html.add_child(folium.Element(progress_html))
    
    return m
    # Add a legend to the map
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 250px; height: 150px; 
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color:white; opacity: 0.8;
                padding: 10px;">
    <b>Legend</b><br>
    <i style="background:lightcoral; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Too Few &nbsp;&nbsp;<br>
    <i style="background:lightgreen; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Balanced &nbsp;&nbsp;<br>
    <i style="background:lightblue; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Too Many &nbsp;&nbsp;<br>
    <i style="background:darkred; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Highly Utilized &nbsp;&nbsp;<br>
    <i style="background:darkgreen; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Well Utilized &nbsp;&nbsp;<br>
    <i style="background:blue; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Under Utilized &nbsp;&nbsp;
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
# Loop through each week and update the map
for week in range(1, 53):
    m = update_map(week)
    clear_output(wait=True)
    display(m)
    time.sleep(1)


LEGEND IS MISSING!!!

In [ ]:
# Create a base map
m1 = folium.Map(location=[df_cumulative_balance['LAT'].mean(), df_cumulative_balance['LNG'].mean()], zoom_start=12)

# Function to determine color based on turnover category
def get_color(turnover_category):
    category_colors = {
        'highly-utilized': 'darkred',
        'healthy': 'green',
        'medium': 'lightgreen',
        'under-utiltized': 'lightgrey',
        'excess-capacity': 'grey'
    }
    return category_colors.get(turnover_category, 'blue')

# Add stations to the map
for _, row in df_cumulative_balance.iterrows():
    # Scale the turnover values to a range between 1 and 20
    scaled_turnover = 1 + (row['TURNOVER'] - df_cumulative_balance['TURNOVER'].min()) * (20 - 1) / (df_cumulative_balance['TURNOVER'].max() - df_cumulative_balance['TURNOVER'].min())
    
    folium.CircleMarker(
        location=[row['LAT'], row['LNG']],
        radius=scaled_turnover,  # Use the scaled turnover for the radius
        color=get_color(row['TO_CATEGORY']),
        fill=True,
        fill_color=get_color(row['TO_CATEGORY']),
        fill_opacity=0.6,
        popup=f"Station: {row['STATION']}\nTurnover: {row['TURNOVER']:.2f}\nCategory: {row['TO_CATEGORY']}"
    ).add_to(m1)

# Create a legend
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: 150px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white; opacity: 0.8;
            ">
&emsp;<b>Legend</b><br>
&emsp;<i class="fa fa-circle" style="color:darkred"></i>&emsp;Highly-utilized<br>
&emsp;<i class="fa fa-circle" style="color:green"></i>&emsp;Healthy<br>
&emsp;<i class="fa fa-circle" style="color:lightgreen"></i>&emsp;Medium<br>
&emsp;<i class="fa fa-circle" style="color:lightgrey"></i>&emsp;Under-utilized<br>
&emsp;<i class="fa fa-circle" style="color:grey"></i>&emsp;Excess-capacity
</div>
'''

m1.get_root().html.add_child(folium.Element(legend_html))

# Display the map
m1


In [ ]:
from geopy.distance import geodesic

# Define the latitude and longitude coordinates of Washington DC
dc_coordinates = (38.907192, -77.046873)  # Adjusted Longitude by 1km to the south

# Function to calculate distance in kilometers
def calculate_distance(lat, lng):
    station_coordinates = (lat, lng)
    return geodesic(dc_coordinates, station_coordinates).kilometers

# Calculate the distance for each station and assign the new columns
df_cumulative_balance['distance_to_centre_km'] = df_cumulative_balance.apply(
    lambda row: round(calculate_distance(row['LAT'], row['LNG']), 1), axis=1
)

# Assign 'inside_outside' based on the distance
df_cumulative_balance['inside_outside'] = df_cumulative_balance['distance_to_centre_km'].apply(
    lambda x: 'in_centre' if x <= 11 else 'outside_centre'
)

df_cumulative_balance

In [ ]:
# Create a pie chart for 'inside_outside' category
plt.figure(figsize=(8, 8))
df_cumulative_balance['inside_outside'].value_counts().plot(
    kind='pie', 
    autopct='%1.1f%%', 
    startangle=140, 
    colors=['green', 'red'], 
    labels=['In Centre', 'Outside Centre']
)

# Set the title
plt.title('Distribution of Stations by Location (Inside/Outside Centre)')
plt.ylabel('')  # Remove the y-axis label for better visualization

# Add a legend with the actual total count for each category
counts = df_cumulative_balance['inside_outside'].value_counts()
legend_labels = [f"In Centre: {counts['in_centre']}", f"Outside Centre: {counts['outside_centre']}"]
plt.legend(legend_labels, loc='upper left', bbox_to_anchor=(1, 0.5))

plt.show()

In [ ]:
# Create a base map centered around Washington DC
m = folium.Map(location=[38.907192, -77.036873], zoom_start=12)

# Add a circle of 15km radius around the center of Washington DC
folium.Circle(
    location=[38.907192, -77.046873], # adjusted coordinate 1km to south
    radius=11000,
    color='blue',
    fill=True,
    fill_opacity=0.1,
    popup='11km Radius'
).add_to(m)

# Function to determine color based on 'inside_outside' column
def get_color(inside_outside):
    return 'green' if inside_outside == 'in_centre' else 'red'

# Add stations to the map
for _, row in df_cumulative_balance.iterrows():
    folium.CircleMarker(
        location=[row['LAT'], row['LNG']],
        radius=5,
        color=get_color(row['inside_outside']),
        fill=True,
        fill_color=get_color(row['inside_outside']),
        fill_opacity=0.6,
        popup=f"Station: {row['STATION']}\nDistance to Center: {row['distance_to_centre_km']} km\nCategory: {row['inside_outside']}"
    ).add_to(m)

# Add a legend to the map
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 200px; height: 90px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white; opacity: 0.8;
            ">
&emsp;<b>Legend</b><br>
&emsp;<i class="fa fa-circle" style="color:green"></i>&emsp;In Centre<br>
&emsp;<i class="fa fa-circle" style="color:red"></i>&emsp;Outside Centre
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
m

In [ ]:
import pandas as pd

# Example of reading a CSV file
df_offices = pd.read_csv('offices.csv')
df_offices

In [ ]:
from geopy.distance import geodesic

# Initialize the 'office_score' column in df_cumulative_balance with 0
df_cumulative_balance['office_score'] = 0.0

# Iterate over each station in df_cumulative_balance
for station_index, station_row in df_cumulative_balance.iterrows():
    station_coords = (station_row['LAT'], station_row['LNG'])
    office_score = 0.0
    
    # Iterate over each office in df_offices
    for office_index, office_row in df_offices.iterrows():
        office_coords = (office_row['Latitude'], office_row['Longitude'])
        
        # Calculate the distance between the station and the office
        distance = geodesic(station_coords, office_coords).meters
        
        # If the distance is less than 500 meters, add the office factor to the office_score
        if distance < 500:
            office_score += office_row['Office_Factor']
    
    # Update the 'office_score' column for the station
    df_cumulative_balance.at[station_index, 'office_score'] = office_score

df_cumulative_balance

In [ ]:
# Sort the dataframe by 'office_score' in descending order and select the top 20 stations
top_20_stations_by_office_score = df_cumulative_balance.sort_values(by='office_score', ascending=False).head(20)

# Display the result
top_20_stations_by_office_score

In [ ]:
from folium import CustomIcon

# Add office locations to the map with custom building icons
for _, office_row in df_offices.iterrows():
    icon = CustomIcon(
        icon_image='building2.png',  # Path to the building image
        icon_size=(30, 30)  # Adjust the size of the icon
    )
    folium.Marker(
        location=[office_row['Latitude'], office_row['Longitude']],
        icon=icon,
        popup=f"Office: {office_row['Office_Location_Name']}<br>"
              f"Category: {office_row['Category']}<br>"
              f"Industry: {office_row['Industry Sector']}"
    ).add_to(m)

# Display the updated map
m


In [ ]:
import pandas as pd

# reading a CSV file
df_metrorail_all = pd.read_csv('Metro_Stations_Regional.csv', sep=',')
df_metrorail_all

In [ ]:
from pyproj import Proj, transform

# Define the projection for the X, Y coordinates (assuming they are in EPSG:3857 - Web Mercator)
# Replace 'EPSG:3857' with the correct projection if different
proj_from = Proj(init='epsg:3857')  # Source projection
proj_to = Proj(init='epsg:4326')    # Target projection (WGS84 - Latitude/Longitude)

# Function to transform X, Y coordinates to latitude and longitude
def transform_coordinates(x, y):
    try:
        lon, lat = transform(proj_from, proj_to, x, y)
        return lat, lon
    except Exception as e:
        print(f"Error transforming coordinates ({x}, {y}): {e}")
        return None, None

# Ensure the columns 'X' and 'Y' exist in df_metrorail_all
if 'X' in df_metrorail_all.columns and 'Y' in df_metrorail_all.columns:
    # Add latitude and longitude columns to df_metrorail_all
    df_metrorail_all[['Latitude', 'Longitude']] = df_metrorail_all.apply(
        lambda row: pd.Series(transform_coordinates(row['X'], row['Y'])), axis=1
    )
else:
    print("Error: The columns 'X' and 'Y' do not exist in df_metrorail_all. Please check the DataFrame.")

# Display the updated DataFrame
df_metrorail_all

In [ ]:
from geopy.distance import geodesic

# Initialize the 'count_metrorail_st_walking_dist' column in df_cumulative_balance with 0
df_cumulative_balance['count_metrorail_st_walking_dist'] = 0

# Iterate over each station in df_cumulative_balance
for station_index, station_row in df_cumulative_balance.iterrows():
    station_coords = (station_row['LAT'], station_row['LNG'])
    count_metrorail_stations = 0
    
    # Iterate over each station in df_metrorail_all
    for _, metro_row in df_metrorail_all.iterrows():
        metro_coords = (metro_row['Latitude'], metro_row['Longitude'])
        
        # Calculate the distance between the station and the metro station
        distance = geodesic(station_coords, metro_coords).meters
        
        # If the distance is less than 500 meters, increment the count
        if distance < 500:
            count_metrorail_stations += 1
    
    # Update the 'count_metrorail_st_walking_dist' column for the station
    df_cumulative_balance.at[station_index, 'count_metrorail_st_walking_dist'] = count_metrorail_stations

df_cumulative_balance

In [ ]:
# Perform a group count for 'count_metrorail_st_walking_dist'
group_count = df_cumulative_balance['count_metrorail_st_walking_dist'].value_counts()

# Display the result
print(group_count)

In [ ]:
from folium import CustomIcon

# Add Metrorail stations to the map with custom metro icons
for _, metro_row in df_metrorail_all.iterrows():
    icon = CustomIcon(
        icon_image='metro_icon.png',  # Path to the metro icon image
        icon_size=(30, 30)  # Adjust the size of the icon
    )
    folium.Marker(
        location=[metro_row['Latitude'], metro_row['Longitude']],
        icon=icon,
        popup=f"Metrorail Station: {metro_row['NAME']}<br>"
            
              
    ).add_to(m)

# Display the updated map
m

In [ ]:
import pandas as pd

# Example of reading a CSV file
df_leisure = pd.read_csv('leisure.csv')
df_leisure

In [ ]:
from geopy.distance import geodesic

# Initialize the 'leisure_score' column in df_cumulative_balance with 0
df_cumulative_balance['leisure_score'] = 0.0

# Iterate over each station in df_cumulative_balance
for station_index, station_row in df_cumulative_balance.iterrows():
    station_coords = (station_row['LAT'], station_row['LNG'])
    leisure_score = 0.0
    
    # Iterate over each leisure location in df_leisure
    for leisure_index, leisure_row in df_leisure.iterrows():
        leisure_coords = (leisure_row['Latitude'], leisure_row['Longitude'])
        
        # Calculate the distance between the station and the leisure location
        distance = geodesic(station_coords, leisure_coords).meters
        
        # If the distance is less than 500 meters, add the leisure factor to the leisure_score
        if distance < 500:
            leisure_score += leisure_row['leisure_factor']
    
    # Update the 'leisure_score' column for the station
    df_cumulative_balance.at[station_index, 'leisure_score'] = leisure_score

df_cumulative_balance

In [ ]:
from folium import CustomIcon

# Add leisure locations to the map with custom leisure icons
for _, leisure_row in df_leisure.iterrows():
    icon = CustomIcon(
        icon_image='leisure.png',  # Path to the leisure icon image
        icon_size=(30, 30)  # Adjust the size of the icon
    )
    folium.Marker(
        location=[leisure_row['Latitude'], leisure_row['Longitude']],
        icon=icon,
        popup=f"Leisure Location: {leisure_row['Leisure_Location']}<br>"
              f"Annual Visitors: {leisure_row['Annual_Visitors']}<br>"
              f"Ranking: {leisure_row['Ranking']}"
    ).add_to(m)

# Display the updated map
m

In [ ]:
import pandas as pd

# Example of reading a CSV file
df_residential = pd.read_csv('residential.csv')
df_residential

In [ ]:
from geopy.distance import geodesic

# Initialize the 'residential_count' column in df_cumulative_balance with 0
df_cumulative_balance['residential_count'] = 0

# Iterate over each station in df_cumulative_balance
for station_index, station_row in df_cumulative_balance.iterrows():
    station_coords = (station_row['LAT'], station_row['LNG'])
    residential_count = 0
    
    # Iterate over each residential location in df_residential
    for residential_index, residential_row in df_residential.iterrows():
        residential_coords = (residential_row['Latitude'], residential_row['Longitude'])
        
        # Calculate the distance between the station and the residential location
        distance = geodesic(station_coords, residential_coords).meters
        
        # If the distance is less than 500 meters, increment the residential_count
        if distance < 500:
            residential_count += 1
    
    # Update the 'residential_count' column for the station
    df_cumulative_balance.at[station_index, 'residential_count'] = residential_count

df_cumulative_balance

In [ ]:
# Perform a group count for 'residential_count'
group_count = df_cumulative_balance['residential_count'].value_counts()

# Display the result
print(group_count)

In [ ]:
from folium import CustomIcon

# Add residential locations to the map with custom residential icons
for _, residential_row in df_residential.iterrows():
    icon = CustomIcon(
        icon_image='residential.png',  # Path to the residential icon image
        icon_size=(30, 30)  # Adjust the size of the icon
    )
    folium.Marker(
        location=[residential_row['Latitude'], residential_row['Longitude']],
        icon=icon,
        popup=f"Area: {residential_row['Area_Name']}<br>"
              f"Population Estimate 2023: {residential_row['Population_Estimate_2023']}"
    ).add_to(m)

# Display the updated map
m


In [ ]:
import pandas as pd

# Example of reading a CSV file
df_venues = pd.read_csv('venues.csv')
df_venues

In [ ]:
from geopy.distance import geodesic

# Initialize the 'venue_count' column in df_cumulative_balance with 0
df_cumulative_balance['venue_count'] = 0

# Iterate over each station in df_cumulative_balance
for station_index, station_row in df_cumulative_balance.iterrows():
    station_coords = (station_row['LAT'], station_row['LNG'])
    venue_count = 0
    
    # Iterate over each venue in df_venues
    for venue_index, venue_row in df_venues.iterrows():
        venue_coords = (venue_row['Latitude'], venue_row['Longitude'])
        
        # Calculate the distance between the station and the venue
        distance = geodesic(station_coords, venue_coords).meters
        
        # If the distance is less than 500 meters, increment the venue_count
        if distance < 500:
            venue_count += 1
    
    # Update the 'venue_count' column for the station
    df_cumulative_balance.at[station_index, 'venue_count'] = venue_count

df_cumulative_balance

In [ ]:
# Perform a group count for 'venue_count'
group_count = df_cumulative_balance['venue_count'].value_counts()

# Display the result
print(group_count)

In [ ]:
from folium import CustomIcon

# Add venue locations to the map with custom venue icons
for _, venue_row in df_venues.iterrows():
    icon = CustomIcon(
        icon_image='venues.png',  # Path to the venue icon image
        icon_size=(30, 30)  # Adjust the size of the icon
    )
    folium.Marker(
        location=[venue_row['Latitude'], venue_row['Longitude']],
        icon=icon,
        popup=f"Venue: {venue_row['Venue_Name']}<br>"
              f"Category: {venue_row['Venue_Category']}<br>"
              f"Annual Customer Visits: {venue_row['Annual_Customer_Visits']}"
    ).add_to(m)

# Display the updated map
m

In [ ]:
# Create a 1x3 grid of subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Define the hue variable and its unique values
hue_var = "inside_outside"
hue_values = df_cumulative_balance[hue_var].unique()
colors = sns.color_palette("tab10", len(hue_values))

# Plot 1: Correlation between apparent temperature and total rides
for hue_value, color in zip(hue_values, colors):
	subset = df_cumulative_balance[df_cumulative_balance[hue_var] == hue_value]
	sns.regplot(data=subset, x="COUNT_PICKUP", y="distance_to_centre_km", ax=axs[0], scatter_kws={'color': color}, line_kws={'color': color}, label=hue_value)
axs[0].set_title("Number of pick-ups vs Station distance to city centre")
axs[0].legend(title=hue_var)

# Plot 2: Correlation between temperature and member rides
for hue_value, color in zip(hue_values, colors):
	subset = df_cumulative_balance[df_cumulative_balance[hue_var] == hue_value]
	sns.regplot(data=subset, x="TURNOVER", y="distance_to_centre_km", ax=axs[1], scatter_kws={'color': color}, line_kws={'color': color}, label=hue_value)
axs[1].set_title("Turnover vs Station distance to city centre")
axs[1].legend(title=hue_var)

# Plot 3: Correlation between windspeed and casual rides
for hue_value, color in zip(hue_values, colors):
	subset = df_cumulative_balance[df_cumulative_balance[hue_var] == hue_value]
	sns.regplot(data=subset, x="IMBALANCE_FACTOR", y="distance_to_centre_km", ax=axs[2], scatter_kws={'color': color}, line_kws={'color': color}, label=hue_value)
axs[2].set_title("Imbalance Factor vs Station distance to city centre")
axs[2].legend(title=hue_var)

# Adjust layout and show the plots
plt.tight_layout()
plt.show()

In [ ]:
# Create a 1x3 grid of subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Plot 1: Correlation between station distance to city centre and number of pick-ups
sns.regplot(data=df_cumulative_balance, x="COUNT_PICKUP", y="distance_to_centre_km", ax=axs[0], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[0].set_title("Number of pick-ups vs Station distance to city centre")

# Plot 2: Correlation between turnover and station distance to city centre
sns.regplot(data=df_cumulative_balance, x="TURNOVER", y="distance_to_centre_km", ax=axs[1], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[1].set_title("Turnover vs Station distance to city centre")

# Plot 3: Correlation between imbalance factor and station distance to city centre
sns.regplot(data=df_cumulative_balance, x="IMBALANCE_FACTOR", y="distance_to_centre_km", ax=axs[2], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[2].set_title("Imbalance Factor vs Station distance to city centre")

# Adjust layout and show the plots
plt.tight_layout()
plt.show()


In [ ]:
# Create a 1x3 grid of subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Plot 1: Correlation between office score and turnover
sns.regplot(data=df_cumulative_balance, x="office_score", y="TURNOVER", ax=axs[0], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[0].set_title("Turnover vs Office Score")
axs[0].set_xlabel("Office Score")
axs[0].set_ylabel("Turnover")

# Plot 2: Correlation between count of metrorail stations within walking distance and turnover
sns.regplot(data=df_cumulative_balance, x="count_metrorail_st_walking_dist", y="TURNOVER", ax=axs[1], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[1].set_title("Turnover vs Count of Metrorail Stations (Walking Distance)")
axs[1].set_xlabel("Count of Metrorail Stations")
axs[1].set_ylabel("Turnover")

# Plot 3: Correlation between leisure score and turnover
sns.regplot(data=df_cumulative_balance, x="leisure_score", y="TURNOVER", ax=axs[2], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[2].set_title("Turnover vs Leisure Score")
axs[2].set_xlabel("Leisure Score")
axs[2].set_ylabel("Turnover")

# Adjust layout and show the plots
plt.tight_layout()
plt.show()

In [ ]:
# Create a 1x3 grid of subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Plot 1: Correlation between office score and imbalance factor
sns.regplot(data=df_cumulative_balance, x="office_score", y="IMBALANCE_FACTOR", ax=axs[0], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[0].set_title("Imbalance Factor vs Office Score")
axs[0].set_xlabel("Office Score")
axs[0].set_ylabel("Imbalance Factor")

# Plot 2: Correlation between count of metrorail stations within walking distance and imbalance factor
sns.regplot(data=df_cumulative_balance, x="count_metrorail_st_walking_dist", y="IMBALANCE_FACTOR", ax=axs[1], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[1].set_title("Imbalance Factor vs Count of Metrorail Stations (Walking Distance)")
axs[1].set_xlabel("Count of Metrorail Stations")
axs[1].set_ylabel("Imbalance Factor")

# Plot 3: Correlation between leisure score and imbalance factor
sns.regplot(data=df_cumulative_balance, x="leisure_score", y="IMBALANCE_FACTOR", ax=axs[2], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[2].set_title("Imbalance Factor vs Leisure Score")
axs[2].set_xlabel("Leisure Score")
axs[2].set_ylabel("Imbalance Factor")

# Adjust layout and show the plots
plt.tight_layout()
plt.show()

In [ ]:
# Create a 2x4 grid of subplots
fig, axs = plt.subplots(2, 4, figsize=(24, 12))

# First row
# Plot 1: Correlation between residential count and number of pick-ups
sns.regplot(data=df_cumulative_balance, x="residential_count", y="COUNT_PICKUP", ax=axs[0, 0], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[0, 0].set_title("Residential Count vs Number of Pick-ups")
axs[0, 0].set_xlabel("Residential Count")
axs[0, 0].set_ylabel("Number of Pick-ups")

# Plot 2: Correlation between residential count and number of returns
sns.regplot(data=df_cumulative_balance, x="residential_count", y="COUNT_RETURN", ax=axs[0, 1], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[0, 1].set_title("Residential Count vs Number of Returns")
axs[0, 1].set_xlabel("Residential Count")
axs[0, 1].set_ylabel("Number of Returns")

# Plot 3: Correlation between residential count and turnover
sns.regplot(data=df_cumulative_balance, x="residential_count", y="TURNOVER", ax=axs[0, 2], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[0, 2].set_title("Residential Count vs Turnover")
axs[0, 2].set_xlabel("Residential Count")
axs[0, 2].set_ylabel("Turnover")

# Plot 4: Correlation between residential count and imbalance factor
sns.regplot(data=df_cumulative_balance, x="residential_count", y="IMBALANCE_FACTOR", ax=axs[0, 3], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[0, 3].set_title("Residential Count vs Imbalance Factor")
axs[0, 3].set_xlabel("Residential Count")
axs[0, 3].set_ylabel("Imbalance Factor")

# Second row
# Plot 1: Correlation between venues count and number of pick-ups
sns.regplot(data=df_cumulative_balance, x="venue_count", y="COUNT_PICKUP", ax=axs[1, 0], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[1, 0].set_title("Venues Count vs Number of Pick-ups")
axs[1, 0].set_xlabel("Venues Count")
axs[1, 0].set_ylabel("Number of Pick-ups")

# Plot 2: Correlation between venues count and number of returns
sns.regplot(data=df_cumulative_balance, x="venue_count", y="COUNT_RETURN", ax=axs[1, 1], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[1, 1].set_title("Venues Count vs Number of Returns")
axs[1, 1].set_xlabel("Venues Count")
axs[1, 1].set_ylabel("Number of Returns")

# Plot 3: Correlation between venues count and turnover
sns.regplot(data=df_cumulative_balance, x="venue_count", y="TURNOVER", ax=axs[1, 2], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[1, 2].set_title("Venues Count vs Turnover")
axs[1, 2].set_xlabel("Venues Count")
axs[1, 2].set_ylabel("Turnover")

# Plot 4: Correlation between venues count and imbalance factor
sns.regplot(data=df_cumulative_balance, x="venue_count", y="IMBALANCE_FACTOR", ax=axs[1, 3], scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
axs[1, 3].set_title("Venues Count vs Imbalance Factor")
axs[1, 3].set_xlabel("Venues Count")
axs[1, 3].set_ylabel("Imbalance Factor")

# Adjust layout and show the plots
plt.tight_layout()
plt.show()

MACHINE LEARNING

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [ ]:
df_cumulative_balance.info()

In [ ]:
df_cumulative_numeric = df_cumulative_balance.select_dtypes(include=['float64', 'int64']).dropna().copy()
df_cumulative_numeric.info()

In [ ]:
df_cumulative_numeric.describe().T

In [ ]:
# Count the number of rows where TURNOVER is greater than 5
count_above_5 = df_cumulative_balance[df_cumulative_balance['TURNOVER'] > 7].shape[0]
print("Number of rows with TURNOVER above 7:", count_above_5)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure with 1 row and 4 columns for the box plots
fig, axs = plt.subplots(1, 4, figsize=(20, 6))

# Boxplot for 'COUNT_PICKUP'
sns.boxplot(data=df_cumulative_balance, y='COUNT_PICKUP', ax=axs[0])
axs[0].set_title('Boxplot of Count Pickup')
axs[0].set_ylabel('COUNT_PICKUP')

# Boxplot for 'COUNT_RETURN'
sns.boxplot(data=df_cumulative_balance, y='COUNT_RETURN', ax=axs[1])
axs[1].set_title('Boxplot of Count Return')
axs[1].set_ylabel('COUNT_RETURN')

# Boxplot for 'TURNOVER'
sns.boxplot(data=df_cumulative_balance, y='TURNOVER', ax=axs[2])
axs[2].set_title('Boxplot of Turnover')
axs[2].set_ylabel('TURNOVER')

# Boxplot for 'IMBALANCE_FACTOR'
sns.boxplot(data=df_cumulative_balance, y='IMBALANCE_FACTOR', ax=axs[3])
axs[3].set_title('Boxplot of Imbalance Factor')
axs[3].set_ylabel('IMBALANCE_FACTOR')

# Adjust layout and show the plots
plt.tight_layout()
plt.show()


In [ ]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df_cumulative_balance['TURNOVER'].quantile(0.25)
Q3 = df_cumulative_balance['TURNOVER'].quantile(0.75)

# Calculate the IQR
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = df_cumulative_balance[df_cumulative_balance['TURNOVER'] > upper_bound]

# Display the outliers
print("Outliers in 'TURNOVER':")
outliers

In [ ]:
# Define a function to calculate outliers for a given column
def calculate_outliers(column_name):
    Q1 = df_cumulative_balance[column_name].quantile(0.25)
    Q3 = df_cumulative_balance[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df_cumulative_balance[(df_cumulative_balance[column_name] < lower_bound) | 
                                 (df_cumulative_balance[column_name] > upper_bound)]

# Calculate outliers for each column
outliers_count_pickup = calculate_outliers('COUNT_PICKUP')
outliers_count_return = calculate_outliers('COUNT_RETURN')
outliers_turnover = calculate_outliers('TURNOVER')
outliers_imbalance_factor = calculate_outliers('IMBALANCE_FACTOR')

# Combine the results into a single dataframe to compare
df_cumulative_balance['is_outlier_count_pickup'] = df_cumulative_balance['STATION'].isin(outliers_count_pickup['STATION'])
df_cumulative_balance['is_outlier_count_return'] = df_cumulative_balance['STATION'].isin(outliers_count_return['STATION'])
df_cumulative_balance['is_outlier_turnover'] = df_cumulative_balance['STATION'].isin(outliers_turnover['STATION'])
df_cumulative_balance['is_outlier_imbalance_factor'] = df_cumulative_balance['STATION'].isin(outliers_imbalance_factor['STATION'])

# Count the number of criteria for which each station is an outlier
df_cumulative_balance['outlier_criteria_count'] = (
    df_cumulative_balance[['is_outlier_count_pickup', 
                            'is_outlier_count_return', 
                            'is_outlier_turnover', 
                            'is_outlier_imbalance_factor']].sum(axis=1)
)

# Count the number of stations that are outliers for 1, 2, 3, or 4 criteria
outlier_counts = df_cumulative_balance['outlier_criteria_count'].value_counts().sort_index()
outlier_counts

# Visualize the results with a legend for outlier reasons
import matplotlib.pyplot as plt

# Define colors for each outlier reason
outlier_colors = {
    'is_outlier_count_pickup': 'blue',
    'is_outlier_count_return': 'orange',
    'is_outlier_turnover': 'green',
    'is_outlier_imbalance_factor': 'red'
}

# Create a figure
plt.figure(figsize=(12, 8))

# Initialize a dictionary to store counts for each number of criteria and reason
criteria_counts = {reason: [] for reason in outlier_colors.keys()}

# Calculate counts for each number of criteria (1, 2, 3, 4) for each reason
for num_criteria in range(1, 5):
    for reason in outlier_colors.keys():
        subset = df_cumulative_balance[
            (df_cumulative_balance[reason]) & 
            (df_cumulative_balance['outlier_criteria_count'] == num_criteria)
        ]
        criteria_counts[reason].append(len(subset))

# Define the bar width and x positions
bar_width = 0.2
x_positions = range(1, 5)

# Plot bars for each outlier reason
for i, (reason, counts) in enumerate(criteria_counts.items()):
    plt.bar(
        [x + i * bar_width for x in x_positions], 
        counts, 
        color=outlier_colors[reason], 
        width=bar_width, 
        label=reason.replace('is_outlier_', '').replace('_', ' ').title()
    )

# Add title and labels
plt.title('Number of Stations Considered Outliers by Number of Criteria')
plt.xlabel('Number of Criteria')
plt.ylabel('Number of Stations')
plt.xticks([x + 1.5 * bar_width for x in x_positions], x_positions)  # Center the x-ticks
plt.legend(title='Outlier Reason')
plt.show()

In [ ]:
# Check for null values
df_cumulative_numeric.isnull().sum()

In [ ]:
pickups = df_cumulative_numeric["COUNT_PICKUP"]
imbalance = df_cumulative_numeric["IMBALANCE_FACTOR"]
turnover = df_cumulative_numeric["TURNOVER"]
distance = df_cumulative_numeric["distance_to_centre_km"]
office = df_cumulative_numeric["office_score"]
metro_count = df_cumulative_numeric["count_metrorail_st_walking_dist"]
leisure = df_cumulative_numeric["leisure_score"]
residential = df_cumulative_numeric["residential_count"]
venues = df_cumulative_numeric["venue_count"]

print(pickups.shape)
print(imbalance.shape)
print(turnover.shape)
print(distance.shape)
print(office.shape)
print(metro_count.shape)
print(leisure.shape)
print(residential.shape)
print(venues.shape)

print(pickups.isnull().sum())
print(imbalance.isnull().sum())
print(turnover.isnull().sum())
print(distance.isnull().sum())
print(office.isnull().sum())
print(metro_count.isnull().sum())
print(leisure.isnull().sum())
print(residential.isnull().sum())
print(venues.isnull().sum())

In [ ]:
# Reshape the dfs to have value for the column instead of a null.

pickups = pickups.values.reshape(-1, 1)
imbalance = imbalance.values.reshape(-1, 1)
turnover = turnover.values.reshape(-1, 1)
distance = distance.values.reshape(-1, 1)
metro_count = metro_count.values.reshape(-1, 1)
leisure = leisure.values.reshape(-1, 1)
residential = residential.values.reshape(-1, 1)
venues = venues.values.reshape(-1, 1)

print(pickups.shape)
print(imbalance.shape)
print(turnover.shape)
print(distance.shape)
print(metro_count.shape)
print(leisure.shape)
print(residential.shape)
print(venues.shape)

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [ ]:
pickups_train, pickups_test, distance_to_centre_train, distance_to_centre_test = train_test_split(pickups, distance_to_centre, test_size=0.2, random_state=420)

In [ ]:
pickups_train

In [ ]:
distance_to_centre_train

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Create a linear regression model
model = linear_model.LinearRegression()

# Fit the model using the training data
model.fit(distance_to_centre_train, pickups_train)

# Predict the values for the test data
predictions = model.predict(distance_to_centre_test)

# Print the model coefficients and intercept
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

# Evaluate the model using R^2 score
r2_score = model.score(distance_to_centre_test, pickups_test)
print("R^2 Score:", r2_score)

# Calculate accuracy metrics
mae = mean_absolute_error(pickups_test, predictions)
mse = mean_squared_error(pickups_test, predictions)
rmse = np.sqrt(mse)

print("Mean Absolute Error (MAE) based on Distance to Centre is:", mae)
print("Mean Squared Error (MSE) based on Distance to Centre is:", mse)
print("Root Mean Squared Error (RMSE) based on Distance to Centre is:", rmse)

# Predict values using test data.
reg_area_score = (model.score(distance_to_centre_test, pickups_test) * 100)
print("The accuracy of the ML model based on Distance to Centre is:", reg_area_score , "%")

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(pickups_test, predictions, color='blue', alpha=0.5, label='Predictions')
plt.scatter(pickups_test, pickups_test, color='red', alpha=0.5, label='Test Data')
plt.legend()
plt.xlabel('Actual Pickups')
plt.ylabel('Predicted Pickups')
plt.title('Comparison of Test Data and Predictions')
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Create a Random Forest Regressor model
model = RandomForestRegressor(random_state=42, n_estimators=100)

# Fit the model using the training data
model.fit(distance_to_centre_train, pickups_train.ravel())

# Predict the values for the test data
predictions = model.predict(distance_to_centre_test)

# Evaluate the model using R^2 score
r2_score = model.score(distance_to_centre_test, pickups_test)
print("R^2 Score:", r2_score)

# Calculate accuracy metrics
mae = mean_absolute_error(pickups_test, predictions)
mse = mean_squared_error(pickups_test, predictions)
rmse = np.sqrt(mse)

print("Mean Absolute Error (MAE) based on Distance to Centre is:", mae)
print("Mean Squared Error (MSE) based on Distance to Centre is:", mse)
print("Root Mean Squared Error (RMSE) based on Distance to Centre is:", rmse)

# Predict values using test data.
reg_area_score = (model.score(distance_to_centre_test, pickups_test) * 100)
print("The accuracy of the ML model based on Distance to Centre is:", reg_area_score , "%")

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(pickups_test, predictions, color='blue', alpha=0.5, label='Predictions')
plt.scatter(pickups_test, pickups_test, color='red', alpha=0.5, label='Test Data')
plt.legend()
plt.xlabel('Actual Pickups')
plt.ylabel('Predicted Pickups')
plt.title('Comparison of Test Data and Predictions')
plt.show()


INSIDE MODEL PERFROMS BETTER WHEN ASKED TO PREDICT distance_to_centre and using number of pickups as indicator
See below

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Create a Random Forest Regressor model
model = RandomForestRegressor(random_state=42, n_estimators=100)

# Fit the model using the training data
model.fit(pickups_train, distance_to_centre_train.ravel())

# Predict the values for the test data
predictions = model.predict(pickups_test)

# Evaluate the model using R^2 score
r2_score = model.score(pickups_test, distance_to_centre_test)
print("R^2 Score:", r2_score)

# Calculate accuracy metrics
mae = mean_absolute_error(distance_to_centre_test, predictions)
mse = mean_squared_error(distance_to_centre_test, predictions)
rmse = np.sqrt(mse)

print("Mean Absolute Error (MAE) based on Number of average daily pickups is:", mae)
print("Mean Squared Error (MSE) based on Number of average daily pickups is:", mse)
print("Root Mean Squared Error (RMSE) based on Number of average daily pickups is:", rmse)

# Predict values using test data.
reg_area_score = (model.score(pickups_test, distance_to_centre_test) * 100)
print("The accuracy of the ML model based on Number of average daily pickups is:", reg_area_score , "%")

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(distance_to_centre_test, predictions, color='blue', alpha=0.5, label='Predictions')
plt.scatter(distance_to_centre_test, distance_to_centre_test, color='red', alpha=0.5, label='Test Data')
plt.legend()
plt.xlabel('Actual Distance to Centre')
plt.ylabel('Predicted Distance to Centre')
plt.title('Comparison of Test Data and Predictions (Model: Random Forest Regressor)')
plt.show()


In [ ]:
# Split the data for training and testing
turnover_train, turnover_test, distance_to_centre_train, distance_to_centre_test = train_test_split(turnover, distance_to_centre, test_size=0.2, random_state=420)

In [ ]:
distance_to_centre_train

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Create a linear regression model
model = linear_model.LinearRegression()

# Fit the model using the training data
model.fit(distance_to_centre_train, turnover_train)

# Predict the values for the test data
predictions = model.predict(distance_to_centre_test)

# Print the model coefficients and intercept
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

# Evaluate the model using R^2 score
r2_score = model.score(distance_to_centre_test, turnover_test)
print("R^2 Score:", r2_score)

# Calculate accuracy metrics
mae = mean_absolute_error(turnover_test, predictions)
mse = mean_squared_error(turnover_test, predictions)
rmse = np.sqrt(mse)

print("Mean Absolute Error (MAE) based on Distance to Centre is:", mae)
print("Mean Squared Error (MSE) based on Distance to Centre is:", mse)
print("Root Mean Squared Error (RMSE) based on Distance to Centre is:", rmse)

# Predict values using test data.
reg_area_score = (model.score(distance_to_centre_test, turnover_test) * 100)
print("The accuracy of the ML model based on Distance to Centre is:", reg_area_score , "%")

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(distance_to_centre_test, predictions, color='blue', alpha=0.5, label='Predictions')
plt.scatter(distance_to_centre_test, turnover_test, color='red', alpha=0.5, label='Test Data')
plt.legend()
plt.xlabel('Distance to Centre')
plt.ylabel('Turnover')
plt.title('Comparison of Test Data and Predictions')
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Create a Random Forest Regressor model
model = RandomForestRegressor(random_state=42, n_estimators=100)

# Fit the model using the training data
model.fit(distance_to_centre_train, turnover_train.ravel())

# Predict the values for the test data
predictions = model.predict(distance_to_centre_test)

# Evaluate the model using R^2 score
r2_score = model.score(distance_to_centre_test, turnover_test)
print("R^2 Score:", r2_score)

# Calculate accuracy metrics
mae = mean_absolute_error(turnover_test, predictions)
mse = mean_squared_error(turnover_test, predictions)
rmse = np.sqrt(mse)

print("Mean Absolute Error (MAE) based on Distance to Centre is:", mae)
print("Mean Squared Error (MSE) based on Distance to Centre is:", mse)
print("Root Mean Squared Error (RMSE) based on Distance to Centre is:", rmse)

# Predict values using test data.
reg_area_score = (model.score(distance_to_centre_test, turnover_test) * 100)
print("The accuracy of the ML model based on Distance to Centre is:", reg_area_score , "%")

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(distance_to_centre_test, predictions, color='blue', alpha=0.5, label='Predictions')
plt.scatter(distance_to_centre_test, turnover_test, color='red', alpha=0.5, label='Test Data')
plt.legend()
plt.xlabel('Distance to Centre')
plt.ylabel('Turnover')
plt.title('Comparison of Test Data and Predictions (Model: Random Forest Regressor)')
plt.show()


MULTIVARIATE Regression

In [ ]:
from sklearn.linear_model import LinearRegression

df_cumulative_numeric

In [ ]:
multivariate = df_cumulative_numeric.drop(['TURNOVER', 'LAT', 'LNG', 'COUNT_PICKUP', 'COUNT_RETURN', 'CUMULATIVE_BALANCE', 'IMBALANCE_FACTOR'], axis='columns')
turnover = df_cumulative_numeric['TURNOVER']

print(multivariate.shape)
print(turnover.shape)

In [ ]:
type(turnover)

In [ ]:
multivariate.columns

In [ ]:
import seaborn as sns

plt.figure(figsize=(12,8))
corr= multivariate.corr()
matrix = np.triu(corr)
sns.heatmap(corr, annot=True, mask=matrix, vmin=-1, vmax=1, cmap='seismic')
plt.show()

In [ ]:
# Reshape the dfs to have value for the column instead of a null.
turnover = turnover.values.reshape(-1, 1)
print(turnover.shape)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

# Split into train and test sets
multivariate_01_train, multivariate_01_test, turnover_train, turnover_test = train_test_split(multivariate, turnover, test_size=0.2, random_state=420)

# Train the model
# Initialize and train the Gradient Boosting Regressor
model = GradientBoostingRegressor(
    random_state=42, 
    n_estimators=48,  # Increase the number of trees
    learning_rate=0.05,  # Lower learning rate for better generalization
    max_depth=5,  # Increase depth for capturing complex patterns
    min_samples_split=125,  # Minimum samples required to split an internal node
    min_samples_leaf=10  # Minimum samples required to be at a leaf node
)
model.fit(multivariate_01_train, turnover_train.ravel())


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Predict values using test data.
reg_multi_score = (model.score(multivariate_01_test, turnover_test) * 100).round(2)
print("The accuracy of the ML model is:", reg_multi_score , "%")

# Predict the values for the test data
predictions = model.predict(multivariate_01_test)

# Calculate evaluation metrics
mae = mean_absolute_error(turnover_test, predictions)
mse = mean_squared_error(turnover_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(turnover_test, predictions)

# Print the evaluation metrics
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R^2 Score:", r2)

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(turnover_test, predictions, color='blue', alpha=0.5, label='Predictions')
plt.scatter(turnover_test, turnover_test, color='red', alpha=0.5, label='Test Data')
plt.legend()
plt.xlabel('Actual Turnover')
plt.ylabel('Predicted Turnover')
plt.title('Comparison of Test Data and Predictions')
plt.show()

# Plot the fitted line
plt.figure(figsize=(10, 6))
sns.regplot(x=turnover_test.flatten(), y=predictions, scatter_kws={'color': 'blue', 'alpha': 0.5}, line_kws={'color': 'red'})
plt.xlabel('Actual Turnover')
plt.ylabel('Predicted Turnover')
plt.title('Fitted Line Plot: Actual vs Predicted Turnover')
plt.show()

# Ensure turnover_test and predictions have the same size
if len(turnover_test) == len(predictions):
	# Calculate residuals
	residuals = turnover_test - predictions

	# Plot Residuals versus Fits
	plt.figure(figsize=(10, 6))
	plt.scatter(predictions, residuals, color='blue', alpha=0.5)
	plt.axhline(y=0, color='red', linestyle='--')
	plt.xlabel('Fitted Values (Predicted Turnover)')
	plt.ylabel('Residuals')
	plt.title('Residuals versus Fits')
	plt.show()
else:
	print("Error: turnover_test and predictions must have the same size.")

Placeholder for continuation

1. Build cross-categories, i.e. TO_CATEGORY and IMBALANCE_BY_AVG_WEEK(OR BY SEASON)
2. Change calculation method for balance to : # return - # pick-up
3. balance categories 
>= 5 then 'too_many_returns'
>1 & <5 then 'skewed_vs_returns'
>=-1 & <=1 then 'balanced'
>-5 & <-1 then 'skewed_vs_pick-ups'
<=-5 then 'too_many_pick-ups'
4. Considerations:

5 subcategories each will create 25 new cross-categories for merge it might be better to consolidate to 3 categories each.

Balance = <-2, -2 to +2, <+2
Logic = if balance > 2 on consistent basis then in one week station is empty or 'clogged'

Turnover <0,5 , 0,5 - 2,5 , >2,5
Logic = 3-4 as upper limit (we want bikes to be used more than once during the day...)




In [ ]:
# to be used

In [ ]:
# Aggregate by station and season the cumulative BALANCE for all days and sort results from lowest to highest value
df_cumulative_balance_season = df_2_unique.groupby(['STATION', 'SEASON', 'LAT', 'LNG'])['BALANCE'].sum().reset_index().sort_values(by='BALANCE')
df_cumulative_balance_season.rename(columns={'BALANCE': 'CUMULATIVE_BALANCE'}, inplace=True)
df_cumulative_balance_season

In [ ]:
import matplotlib.pyplot as plt

# Define the order of seasons and their corresponding colors
season_order = ['Winter', 'Spring', 'Summer', 'Fall']
season_colors = {'Winter': 'blue', 'Spring': 'green', 'Summer': 'orange', 'Fall': 'brown'}

# Select the top 25 stations with the highest cumulative balance
top_25_stations = df_cumulative_balance.nlargest(25, 'CUMULATIVE_BALANCE')['STATION']

# Filter the df_cumulative_balance_season DataFrame to include only the top 25 stations
df_top_25_season = df_cumulative_balance_season[df_cumulative_balance_season['STATION'].isin(top_25_stations)]

# Pivot the DataFrame to have seasons as columns and stations as rows
df_pivot = df_top_25_season.pivot(index='STATION', columns='SEASON', values='CUMULATIVE_BALANCE')

# Reindex the columns to ensure the correct order of seasons
df_pivot = df_pivot[season_order]

# Sort the DataFrame by the cumulative balance per year
df_pivot = df_pivot.loc[top_25_stations]

# Plot the data with reversed season order
ax = df_pivot[season_order[::-1]].plot(kind='barh', figsize=(10, 10), color=[season_colors[season] for season in season_order[::-1]])
plt.title('Top 25 Stations with Highest Cumulative Balance by Season')
plt.xlabel('Cumulative Balance')
plt.ylabel('Station')
plt.legend(title='Season')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Define the order of seasons and their corresponding colors
season_order = ['Winter', 'Spring', 'Summer', 'Fall']
season_colors = {'Winter': 'blue', 'Spring': 'green', 'Summer': 'orange', 'Fall': 'brown'}

# Select the top 30 stations with the highest cumulative balance
top_30_stations = df_cumulative_balance.nlargest(30, 'CUMULATIVE_BALANCE')['STATION']

# Filter the df_cumulative_balance_season DataFrame to include only the top 25 stations
df_top_30_season = df_cumulative_balance_season[df_cumulative_balance_season['STATION'].isin(top_30_stations)]

# Pivot the DataFrame to have seasons as columns and stations as rows
df_pivot = df_top_30_season.pivot(index='STATION', columns='SEASON', values='CUMULATIVE_BALANCE')

# Reindex the columns to ensure the correct order of seasons
df_pivot = df_pivot[season_order]

# Sort the DataFrame by the cumulative balance per year
df_pivot = df_pivot.loc[top_30_stations]

# Plot the data with reversed season order
ax = df_pivot[season_order[::-1]].plot(kind='barh', figsize=(10, 10), color=[season_colors[season] for season in season_order[::-1]])
plt.title('Top 30 Stations with Highest Cumulative Balance by Season')
plt.xlabel('Cumulative Balance')
plt.ylabel('Station')
plt.legend(title='Season')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Define the order of seasons and their corresponding colors
season_order = ['Winter', 'Spring', 'Summer', 'Fall']
season_colors = {'Winter': 'blue', 'Spring': 'green', 'Summer': 'orange', 'Fall': 'brown'}

# Select the 30 stations with the smallest cumulative balance
bottom_30_stations = df_cumulative_balance.nsmallest(30, 'CUMULATIVE_BALANCE')['STATION']

# Filter the df_cumulative_balance_season DataFrame to include only the bottom 30 stations
df_bottom_30_season = df_cumulative_balance_season[df_cumulative_balance_season['STATION'].isin(bottom_30_stations)]

# Pivot the DataFrame to have seasons as columns and stations as rows
df_pivot = df_bottom_30_season.pivot(index='STATION', columns='SEASON', values='CUMULATIVE_BALANCE')

# Reindex the columns to ensure the correct order of seasons
df_pivot = df_pivot[season_order]

# Sort the DataFrame by the cumulative balance per year
df_pivot = df_pivot.loc[bottom_30_stations]

# Plot the data with reversed season order
ax = df_pivot[season_order[::-1]].plot(kind='barh', figsize=(10, 10), color=[season_colors[season] for season in season_order[::-1]])
plt.title('Bottom 30 Stations with Smallest Cumulative Balance by Season')
plt.xlabel('Cumulative Balance')
plt.ylabel('Station')
plt.legend(title='Season')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Define the order of seasons and their corresponding colors
season_order = ['Winter', 'Spring', 'Summer', 'Fall']
season_colors = {'Winter': 'blue', 'Spring': 'green', 'Summer': 'orange', 'Fall': 'brown'}

# Select the top 30 and bottom 30 stations with the highest and smallest cumulative balance
top_30_stations = df_cumulative_balance.nlargest(30, 'CUMULATIVE_BALANCE')['STATION']
bottom_30_stations = df_cumulative_balance.nsmallest(30, 'CUMULATIVE_BALANCE')['STATION']

# Filter the df_cumulative_balance_season DataFrame to include only the top 30 and bottom 30 stations
df_top_30_season = df_cumulative_balance_season[df_cumulative_balance_season['STATION'].isin(top_30_stations)]
df_bottom_30_season = df_cumulative_balance_season[df_cumulative_balance_season['STATION'].isin(bottom_30_stations)]

# Combine the top 30 and bottom 30 dataframes
df_combined_season = pd.concat([df_top_30_season, df_bottom_30_season])

# Pivot the DataFrame to have seasons as columns and stations as rows
df_pivot = df_combined_season.pivot(index='STATION', columns='SEASON', values='CUMULATIVE_BALANCE')

# Reindex the columns to ensure the correct order of seasons
df_pivot = df_pivot[season_order]

# Sort the DataFrame by the cumulative balance per year
df_pivot = df_pivot.loc[bottom_30_stations.tolist() + top_30_stations.tolist()]

# Plot the data with reversed season order
fig, ax1 = plt.subplots(figsize=(12, 15))

# Plot bottom 30 stations
df_pivot.loc[bottom_30_stations, season_order[::-1]].plot(kind='barh', ax=ax1, color=[season_colors[season] for season in season_order[::-1]], legend=False)
ax1.set_ylabel('Bottom 30 Stations')
ax1.set_xlabel('Cumulative Balance')

# Create a second y-axis for the top 30 stations
ax2 = ax1.twinx()
df_pivot.loc[top_30_stations, season_order[::-1]].plot(kind='barh', ax=ax2, color=[season_colors[season] for season in season_order[::-1]], legend=False)
ax2.set_ylabel('Top 30 Stations')
ax2.set_yticks(ax1.get_yticks())
ax2.set_yticklabels(top_30_stations)

# Add a vertical line at X-Axis = 0
ax1.axvline(x=0, color='black', linestyle='--', linewidth=1)

# Set the title and legend
plt.title('Top 30 and Bottom 30 Stations with Cumulative Balance by Season')
ax1.legend(title='Season', loc='upper left', bbox_to_anchor=(1.05, 1))
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import folium
from folium import plugins

In [ ]:
# This map uses adjusted scale for each season

# Create a base map
m4 = folium.Map(location=[df_cumulative_balance_season['LAT'].mean(), df_cumulative_balance_season['LNG'].mean()], zoom_start=12)

# Function to determine color based on balance
def get_color(balance):
    if balance > 0:
        return 'blue'
    else:
        return 'red'

# Function to add stations to a feature group
def add_stations_to_group(group, stations, color, max_balance):
    for _, row in stations.iterrows():
        folium.CircleMarker(
            location=[row['LAT'], row['LNG']],
            radius=1 + (abs(row['CUMULATIVE_BALANCE']) / max_balance) * 10,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=f"Station: {row['STATION']}\nBalance: {row['CUMULATIVE_BALANCE']}"
        ).add_to(group)

# Create feature groups for each season
seasons = ['Winter', 'Spring', 'Summer', 'Fall']
feature_groups = {season: folium.FeatureGroup(name=season, show=(season == 'Summer')) for season in seasons}

# Add stations to each feature group
for season in seasons:
    season_stations = df_cumulative_balance_season[df_cumulative_balance_season['SEASON'] == season]
    
    # Create dataframes for top 50, bottom 50, and the rest
    top_50_stations = season_stations.nlargest(50, 'CUMULATIVE_BALANCE')
    bottom_50_stations = season_stations.nsmallest(50, 'CUMULATIVE_BALANCE')
    remaining_stations = season_stations[~season_stations['STATION'].isin(top_50_stations['STATION']) & ~season_stations['STATION'].isin(bottom_50_stations['STATION'])]
    
    # Calculate max balance for scaling
    max_balance = max(abs(season_stations['CUMULATIVE_BALANCE'].max()), abs(season_stations['CUMULATIVE_BALANCE'].min()))
    
    # Add stations to the feature group
    add_stations_to_group(feature_groups[season], top_50_stations, 'blue', max_balance)
    add_stations_to_group(feature_groups[season], bottom_50_stations, 'red', max_balance)
    add_stations_to_group(feature_groups[season], remaining_stations, 'green', max_balance)

# Add feature groups to the map
for season, group in feature_groups.items():
    group.add_to(m4)

# Add layer control to switch between seasons
folium.LayerControl(collapsed=False).add_to(m4)

# Display the map
m4

In [ ]:
# This map uses same scale for all seasons


# Create a base map
m5 = folium.Map(location=[df_cumulative_balance_season['LAT'].mean(), df_cumulative_balance_season['LNG'].mean()], zoom_start=12)

# Function to determine color based on balance
def get_color(balance):
    if balance > 0:
        return 'blue'
    else:
        return 'red'

# Function to add stations to a feature group
def add_stations_to_group(group, stations, color, max_balance):
    for _, row in stations.iterrows():
        folium.CircleMarker(
            location=[row['LAT'], row['LNG']],
            radius=1 + (abs(row['CUMULATIVE_BALANCE']) / max_balance) * 10,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=f"Station: {row['STATION']}\nBalance: {row['CUMULATIVE_BALANCE']}"
        ).add_to(group)

# Create feature groups for each season
seasons = ['Winter', 'Spring', 'Summer', 'Fall']
feature_groups = {season: folium.FeatureGroup(name=season, show=(season == 'Summer')) for season in seasons}

# Add stations to each feature group
for season in seasons:
    season_stations = df_cumulative_balance_season[df_cumulative_balance_season['SEASON'] == season]
    
    # Create dataframes for top 50, bottom 50, and the rest
    top_50_stations = season_stations.nlargest(50, 'CUMULATIVE_BALANCE')
    bottom_50_stations = season_stations.nsmallest(50, 'CUMULATIVE_BALANCE')
    remaining_stations = season_stations[~season_stations['STATION'].isin(top_50_stations['STATION']) & ~season_stations['STATION'].isin(bottom_50_stations['STATION'])]
    
    # Calculate max balance for scaling
    max_balance = max(abs(df_cumulative_balance_season['CUMULATIVE_BALANCE'].max()), abs(df_cumulative_balance_season['CUMULATIVE_BALANCE'].min()))
    
    # Add stations to the feature group
    add_stations_to_group(feature_groups[season], top_50_stations, 'blue', max_balance)
    add_stations_to_group(feature_groups[season], bottom_50_stations, 'red', max_balance)
    add_stations_to_group(feature_groups[season], remaining_stations, 'green', max_balance)

# Add feature groups to the map
for season, group in feature_groups.items():
    group.add_to(m5)

# Add layer control to switch between seasons
folium.LayerControl(collapsed=False).add_to(m5)

# Display the map
m5

In [ ]:
# Aggregate by station and weekday the cumulative BALANCE for all days and sort results from lowest to highest value
df_cumulative_balance_weekday = df_2_unique.groupby(['STATION', 'WEEKDAY', 'LAT', 'LNG'])['BALANCE'].sum().reset_index().sort_values(by='BALANCE')
df_cumulative_balance_weekday.rename(columns={'BALANCE': 'CUMULATIVE_BALANCE'}, inplace=True)
df_cumulative_balance_weekday

In [ ]:
# This map uses same scale for all weekdays

# Create a base map
m6 = folium.Map(location=[df_cumulative_balance_weekday['LAT'].mean(), df_cumulative_balance_weekday['LNG'].mean()], zoom_start=12)

# Function to determine color based on balance
def get_color(balance):
    if balance > 0:
        return 'blue'
    else:
        return 'red'

# Function to add stations to a feature group
def add_stations_to_group(group, stations, color, max_balance):
    for _, row in stations.iterrows():
        folium.CircleMarker(
            location=[row['LAT'], row['LNG']],
            radius=1 + (abs(row['CUMULATIVE_BALANCE']) / max_balance) * 10,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=f"Station: {row['STATION']}\nBalance: {row['CUMULATIVE_BALANCE']}"
        ).add_to(group)

# Create feature groups for each weekday
weekdays = ['Yes', 'No']
feature_groups = {weekday: folium.FeatureGroup(name=weekday, show=(weekday == 'Yes')) for weekday in weekdays}

# Add stations to each feature group
for weekday in weekdays:
    weekday_stations = df_cumulative_balance_weekday[df_cumulative_balance_weekday['WEEKDAY'] == weekday]
    
    # Create dataframes for top 50, bottom 50, and the rest
    top_50_stations = weekday_stations.nlargest(50, 'CUMULATIVE_BALANCE')
    bottom_50_stations = weekday_stations.nsmallest(50, 'CUMULATIVE_BALANCE')
    remaining_stations = weekday_stations[~weekday_stations['STATION'].isin(top_50_stations['STATION']) & ~weekday_stations['STATION'].isin(bottom_50_stations['STATION'])]
    
    # Calculate max balance for scaling
    max_balance = max(abs(df_cumulative_balance_weekday['CUMULATIVE_BALANCE'].max()), abs(df_cumulative_balance_weekday['CUMULATIVE_BALANCE'].min()))
    
    # Add stations to the feature group
    add_stations_to_group(feature_groups[weekday], top_50_stations, 'blue', max_balance)
    add_stations_to_group(feature_groups[weekday], bottom_50_stations, 'red', max_balance)
    add_stations_to_group(feature_groups[weekday], remaining_stations, 'green', max_balance)

# Add feature groups to the map
for weekday, group in feature_groups.items():
    group.add_to(m6)

# Add layer control to switch between weekdays
folium.LayerControl(collapsed=False).add_to(m6)

# Display the map
m6


In [ ]:
# Use df_2_unique and add to each line 'member_type'